In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
import utils.risktool as risktool
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.utils import load_data_from_csv, print_model

# restore saved variables
# %store -r summary_general2_FL_interpret

Current working directory is now:  D:\github\interpretable-machine-learning


### CART ~~& EBM~~

In [2]:
data = pd.read_csv("./broward/data/broward_data.csv").sort_values('person_id')
x = data.loc[:,:'five_year']
y = data['general_two_year'].values

In [3]:

print("Part 1 done.")

#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003, 0.004, 0.005]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         impurity=impurity, 
                         seed = 816)

print("CART: ", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs']))


#### EBM
estimators = [40,60,80,100]
depth = [1,2,3]
learning_rate = [0.01]
validation_size = [0.7, 0.9]
#ebm_summary = interpret.EBM(X=x,
#                            Y=y,
#                            learning_rate = learning_rate,
#                            depth = depth,
#                            estimators=estimators,
#                            validation_size=validation_size,
#                            seed=816)




Part 1 done.


D:\github\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
D:\github\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
D:\github\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
D:\github\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
D:\github\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)


CART:  0.6127212816209843 0.04542192308852784


In [4]:
#print("CART: ", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs']))
#print("EMB: ", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs']))

---

### Lasso Stumps

In [5]:
## load stumps & original data
data = pd.read_csv("./broward/data/broward_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("./broward/data/broward_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on=['person_id', 'screening_date'])

## split X & Y
X_stumps = data.loc[:,:'five_year1'].copy()
Y_stumps = data['general_two_year'].values.copy()
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test stumps data
train_stumps = pd.read_csv("./broward/data/broward_train_stumps.csv").sort_values('person_id')
test_stumps = pd.read_csv("./broward/data/broward_test_stumps.csv").sort_values('person_id')
X_train_stumps = train_stumps.loc[:,:'five_year1'].copy()
X_test_stumps = test_stumps.loc[:,:'five_year1'].copy()
Y_train_stumps = train_stumps['general_two_year'].values.copy()
Y_test_stumps = test_stumps['general_two_year'].values.copy()
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

In [6]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.1, 
                                        columns=cols, 
                                        seed=816)
## unique original features
unique_stumps = []
for i in single_stump_model['features']:
    unique_stumps.append(''.join([j for j in i if not j.isdigit()]))
print(len(np.unique(unique_stumps)))

14


In [7]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05,
                                              0.06, 0.07, 0.08, 0.09, 0.1]}, 
                                seed = 816)

D:\github\interpretable-machine-learning\utils\fairness_functions.py:47: RuntimeWarning: invalid value encountered in longlong_scalars
  npv = tn / (tn + fn) if (tn+fn) is not 0 else 0
D:\github\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
D:\github\interpretable-machine-learning\utils\fairness_functions.py:47: RuntimeWarning: invalid value encountered in longlong_scalars
  npv = tn / (tn + fn) if (tn+fn) is not 0 else 0
D:\github\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
D:\github\interpretable-machine-learning\utils\fairness_functions.py:47: RuntimeWarning: invalid value encountered in longlong_scalars
  npv = tn / (tn + fn) if (tn+fn) is not 0 else 0
D:\github\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value 

In [8]:
print("CART: ", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs']))
#print("EMB: ", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs']))
print("Additive:", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs']))

CART:  0.6127212816209843 0.04542192308852784
Additive: 0.6510913175121672 0.036576691344414325


---

### RiskSLIM

In [9]:
## load stumps data
data = pd.read_csv("./broward/data/broward_stumps.csv").sort_values('person_id')

## single stump model
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.05,  
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

11

In [10]:
### Subset features
selected_features = single_stump_model['features']
if 'sex1' in selected_features:
    selected_features = ['general_two_year', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['general_two_year', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0
    
sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on=['person_id', 'screening_date'])
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [11]:
riskslim_cs_summary = slim.risk_nested_cv_constrain(X=sub_X, 
                                                    Y=sub_Y, 
                                                    indicator=indicator,
                                                    y_label='general_two_year', 
                                                    max_coef=5,
                                                    max_coef_number=5, 
                                                    max_runtime=1000, 
                                                    c=1e-6, 
                                                    max_offset=100,
                                                    seed=816)

setting c0_value = 0.0 for (Intercept) to ensure that intercept is not penalized
11/01/21 @ 04:52 PM | 223 rows in lookup table
11/01/21 @ 04:52 PM | ------------------------------------------------------------
11/01/21 @ 04:52 PM | runnning initialization procedure
11/01/21 @ 04:52 PM | ------------------------------------------------------------
11/01/21 @ 04:52 PM | CPA produced 2 cuts
11/01/21 @ 04:52 PM | running naive rounding on 24 solutions
11/01/21 @ 04:52 PM | best objective value: 0.6380
11/01/21 @ 04:52 PM | rounding produced 5 integer solutions
11/01/21 @ 04:52 PM | best objective value is 0.6470
11/01/21 @ 04:52 PM | running sequential rounding on 24 solutions
11/01/21 @ 04:52 PM | best objective value: 0.6380
11/01/21 @ 04:52 PM | sequential rounding produced 6 integer solutions
11/01/21 @ 04:52 PM | best objective value: 0.6470
11/01/21 @ 04:52 PM | polishing 11 solutions
11/01/21 @ 04:52 PM | best objective value: 0.6470
11/01/21 @ 04:52 PM | polishing produced 5 integ

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/01/21 @ 04:52 PM | adding 117 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6470.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt   

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/01/21 @ 04:52 PM | adding 123 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6572.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt   

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/01/21 @ 04:52 PM | adding 126 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6492.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt   

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/01/21 @ 04:52 PM | adding 126 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6420.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt   

...

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/01/21 @ 04:53 PM | adding 130 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6586.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt   

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/01/21 @ 04:53 PM | adding 121 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6588.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt   

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/01/21 @ 04:53 PM | adding 148 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6559.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt   

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/01/21 @ 04:53 PM | adding 135 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6585.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt   

D:\github\interpretable-machine-learning\utils\fairness_functions.py:44: RuntimeWarning: divide by zero encountered in longlong_scalars
  ratio = fn / fp if fp is not 0 else 0
D:\github\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)


In [12]:
print(np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs']))
#print(np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs']))
print(np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs']))
print(np.mean(riskslim_cs_summary['test_auc']))

0.6127212816209843 0.04542192308852784
0.6510913175121672 0.036576691344414325
0.6238526664057629


#### Single RiskSLIM Model 

In [13]:
selected_features = ["general_two_year"] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

## new_train_data
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'general_two_year',
    'sample_weights': sample_weights
}

In [14]:
model_info, mip_info, lcpa_info = slim.risk_slim_constrain(new_train_data, 
                                                           max_coefficient=5, 
                                                           max_L0_value=5, 
                                                           c0_value=1e-6, 
                                                           max_offset=100, 
                                                           max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0_value = 0.0 for (Intercept) to ensure that intercept is not penalized
11/01/21 @ 04:53 PM | 223 rows in lookup table
11/01/21 @ 04:53 PM | ------------------------------------------------------------
11/01/21 @ 04:53 PM | runnning initialization procedure
11/01/21 @ 04:53 PM | ------------------------------------------------------------
11/01/21 @ 04:53 PM | CPA produced 2 cuts
11/01/21 @ 04:53 PM | running naive rounding on 28 solutions
11/01/21 @ 04:53 PM | best objective value: 0.6490
11/01/21 @ 04:53 PM | rounding produced 5 integer solutions
11/01/21 @ 04:53 PM | best objective value is 0.6605
11/01/21 @ 04:53 PM | running sequential rounding on 28 solutions
11/01/21 @ 04:53 PM | best objective value: 0.6490
11/01/21 @ 04:53 PM | sequential rounding produced 5 integer solutions
11/01/21 @ 04:53 PM | best objective value: 0.6579
11/01/21 @ 04:53 PM | polishing 10 solutions
11/01/21 @ 04:53 PM | best objective value: 0.6579
11/01/21 @ 04:53 PM | polishing produced 5 integ

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/01/21 @ 04:53 PM | adding 133 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.6579.
Tried aggregator 1 time.
Reduced MIP has 13 rows, 26 columns, and 47 nonzeros.
Reduced MIP has 11 binaries, 13 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt   

In [15]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

print("Train AUC: {}".format(roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))))
print("Test AUC: {}".format(roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))))

Train AUC: 0.6470807592797929
Test AUC: 0.6463295355415157


---

### Arnold PSA

In [16]:
### load data
data = pd.read_csv("./broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 
                       'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['general_two_year'].values

## test model
arnold = risktool.risktool(X_arnold, Y_arnold, label='arnold_nca')
print(np.mean(arnold['auc']))

0.6047759054917025


#### Single Arnold PSA

In [17]:
test_data = pd.read_csv("./broward/data/broward_arnold_test.csv")
X = test_data['arnold_nca'].values
Y = test_data['general_two_year'].values
roc_auc_score(Y, X)

0.6426154537524122

---

### Compass

In [18]:
### load data
data = pd.read_csv("./broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['Risk of Recidivism_decile_score', 'sex', 'race', 'person_id', 
                       'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['general_two_year'].values

## test model
compas = risktool.risktool(X_arnold, Y_arnold, label='Risk of Recidivism_decile_score')
print(np.mean(compas['auc']))

0.631444326887201


### Results

In [20]:
#### save results
summary_general2_FL_interpret = {"cart": cart_summary,
                                 #"ebm": ebm_summary, 
                                 'stumps': stump_summary, 
                                 'riskslim': riskslim_cs_summary, 
                                 'arnold': arnold, 
                                 'compas': compas}
#%store summary_general2_FL_interpret

In [21]:
results = [["cart", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           #["ebm", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['riskslim', np.mean(riskslim_cs_summary['test_auc'])],
           ['arnold', np.mean(arnold['auc'])], 
           ['compas', np.mean(compas['auc'])]]
results

[['cart', 0.6127212816209843, 0.04542192308852784],
 ['stumps', 0.6510913175121672, 0.036576691344414325],
 ['riskslim', 0.6238526664057629],
 ['arnold', 0.6047759054917025],
 ['compas', 0.631444326887201]]

In [22]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       #np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_cs_summary['test_auc']),]

In [23]:
path = "./broward/logs/interpretable/"
results = [["", "CART",
            #"EBM", 
            "Lasso Stumps", "RiskSLIM", "Performance Range", "Arnold PSA", "Compas"],
           ["General", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            #np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_cs_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_cs_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold['auc']), 3)) + " (" + np.str(round(np.std(arnold['auc']),3)) + ")", 
            np.str(round(np.mean(compas['auc']), 3)) + " (" + np.str(round(np.std(compas['auc']),3)) + ")"]]
with open(path + 'FL-two-year-interpretable-summary.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

D:\github\interpretable-machine-learning\env\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """
D:\github\interpretable-machine-learning\env\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
D:\github\interpretable-machine-learning\env\lib\site-packages\ipykernel_la

### Save Fairness Matrix

In [24]:
matrix = ['confusion_matrix_stats', 'calibration_stats', 'race_auc', 'condition_pn', 'no_condition_pn']
name = ['confusion', 'calibration', 'race-auc', 'condition-pn', 'no-condition-pn']

In [26]:
for i in range(len(matrix)):
    
    if ( (i == 0) | (i == 1) ):
        
        path = './broward/fairness/' + name[i] + '/two-year/general/'
        ## confusion matrix and calibration only applies to cart, ebm, riskslim and stumps
        cart_matrix = cart_summary[matrix[i]]
        #ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_cs_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        #ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        
    else:
        path = './broward/fairness/' + name[i] + '/two-year/general/'
        
        ## including arnold and compas now
        cart_matrix = cart_summary[matrix[i]]
        #ebm_matrix = ebm_summary[matrix[i]]
        riskslim_matrix = riskslim_cs_summary[matrix[i]]
        stumps_matrix = stump_summary[matrix[i]]
        arnold_matrix = arnold[matrix[i]]
        compas_matrix = compas[matrix[i]]
        
        ## save results
        cart_matrix.to_csv(path+'cart-'+name[i]+'.csv', index=None,header=True)
        #ebm_matrix.to_csv(path+'ebm-'+name[i]+'.csv', index=None,header=True)
        riskslim_matrix.to_csv(path+'riskslim-'+name[i]+'.csv', index=None,header=True)
        stumps_matrix.to_csv(path+'stumps-'+name[i]+'.csv', index=None,header=True)
        arnold_matrix.to_csv(path+'arnold-'+name[i]+'.csv', index=None,header=True)
        compas_matrix.to_csv(path+'compas-'+name[i]+'.csv', index=None,header=True)